<h1><center>Metody uczenia maszynowego</center></h1>
<h3><center><ul style="list-style: none;">
    <li>Patrycja Szczakowska ???</li>
    <li>Maciej Majchrowski ???</li>
    <li>Bartosz Janicki ???</li>
    <li>Mateusz Gałasiński 234054</li>
    <li>Jakub Kurek 234078</li>
    </ul></center></h3> 
<h2><center>Zadanie 2 - Klastryzacja danych - uczenie nienadzorowane</center></h2>

### Zależności
*Dodatkowe zewnętrzne biblioteki, instalowane dzięki IPython "magic commands"*

In [ ]:
# install additional packages
!pip install k3d > /dev/null
!pip install fuzzy-c-means > /dev/null

### Imports

In [ ]:
from pathlib import Path
import matplotlib
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0}) # Do not warn about high memory consumption
from pandas import read_csv
import json
%matplotlib notebook

In [ ]:
import sklearn
from uuid import uuid4
from sklearn import datasets
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
from sklearn.metrics import calinski_harabasz_score, pairwise_distances, davies_bouldin_score, adjusted_rand_score, adjusted_mutual_info_score, homogeneity_score, completeness_score, homogeneity_completeness_v_measure, v_measure_score, silhouette_score
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, OPTICS, cluster_optics_dbscan
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import numpy as np
from fcmeans import FCM

## Konfiguracja

In [ ]:
class Config(object):
    def __init__(self, dataSourceUrl, dataDelimiter, n_clusters, eps, min_samples, xi, min_cluster_size, m, ndims):
        self.dataSourceUrl = dataSourceUrl
        self.dataDelimiter = dataDelimiter
        self.n_clusters = n_clusters
        self.eps = eps
        self.min_samples = min_samples
        self.xi = xi
        self.min_cluster_size = min_cluster_size
        self.m = m
        self.ndims = ndims
def as_config(dct):
    return Config(
        dct['dataSourceUrl'],
        dct['dataDelimiter'],
        dct['n_clusters'],
        dct['eps'],
        dct['min_samples'],
        dct['xi'],
        dct['min_cluster_size'],
        dct['m'],
        dct['ndims'],
    )

## Wczytanie zbioru danych

In [ ]:
# from pandas.plotting import scatter_matrix

def load_dataset():
    # show 5 rows from data
    for line in [line for line in Path(config.dataSourceUrl).read_text().splitlines()[:5] if line]:
        print(line)

    global dataset
    dataset = read_csv(config.dataSourceUrl, delimiter=config.dataDelimiter, header=0)
    print(f"Data set shape: {dataset.shape}")
    dataset.describe()

In [ ]:
from math import sqrt
def annot_elbow(x,y,ax=None):
    arg = x[0]
    val = y[0]
    for i, a in enumerate(x):
        if (a+2)*sqrt(y[i]) < (arg+2)*sqrt(val):
            arg = a
            val = y[i]
    val = np.argmin(y)
    text= "x={:.3f}, y={:.3f}".format(arg, val)
    if not ax:
        ax=plt.gca()
    bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
    arrowprops=dict(arrowstyle="->")
    kw = dict(xycoords='data',textcoords="axes fraction",
              arrowprops=arrowprops,
              bbox=bbox_props, ha="right", va="top")
    ax.annotate(text, xy=(arg, val), xytext=(0.5,-0.05), **kw)

In [ ]:
def scatter3d(x, labels):
    fig = plt.figure(figsize=(12, 6), dpi=60)
    ax = plt.axes(projection="3d")
    ax.scatter3D(x[:, 0:1], x[:, 1:2], x[:, 2:3], c=labels, cmap='inferno');
    plt.show()

In [ ]:
def classify_each(x, labels_true=None):
    models = []
    models.extend([
        (f"AgglomerativeClustering-{linkage}", AgglomerativeClustering(linkage=linkage, n_clusters=config.n_clusters))
        for linkage in ['ward', 'average', 'complete', 'single']])
    models.extend([
        (f"GaussianMixture-{covariance_type}", GaussianMixture(n_components=config.n_clusters, covariance_type=covariance_type))
        for covariance_type in ['full', 'tied', 'diag', 'spherical']])
    models.extend([
        ('DBSCAN', DBSCAN(eps=config.eps, min_samples=config.min_samples)),
        ('OPTICS', OPTICS(min_samples=config.min_samples, xi=config.xi, min_cluster_size=config.min_cluster_size)),
        ('KNN', KMeans(n_clusters=config.n_clusters, random_state=1)),
        ('FCM', FCM(n_clusters=config.n_clusters, m=config.m))
    ])

    results = []
    names = []
    for i, (name, model) in enumerate(models):
        print(f"---------------------------\nRunning clustering for: {name}")
        labels = []
        if 'GaussianMixture' in name:
            labels = model.fit_predict(x)
        elif 'OPTICS' in name:
            clustering = model.fit(x)
            labels = cluster_optics_dbscan(reachability=clustering.reachability_,
                                       core_distances=clustering.core_distances_,
                                       ordering=clustering.ordering_, eps=config.eps)
        elif 'FCM' in name:
            clustering = model.fit(x)
            labels  = model.u.argmax(axis=1)
        else:
            clustering = model.fit(x)
            labels=clustering.labels_
    ### Results 
        if labels_true:
            print("Homogeneity: %0.3f" % homogeneity_score(labels_true, labels))
            print("Completeness: %0.3f" % completeness_score(labels_true, labels))
            print("V-measure: %0.3f" % v_measure_score(labels_true, labels))
            print("Adjusted Rand Index: %0.3f" % adjusted_rand_score(labels_true, labels))
            print("Adjusted Mutual Information: %0.3f" % adjusted_mutual_info_score(labels_true, labels))
        if len(set(labels)) > 1:
            print(f"Quality: {davies_bouldin_score(x, labels)}")
            print("Silhouette Coefficient: %0.3f" % silhouette_score(x, labels))
        print(f"n_clusters_: {len(set(labels)) - (1 if -1 in labels else 0)}")
        print(f"n_noise_: {list(labels).count(-1)}")

    ### Plots
        if config.ndims == 3:
            scatter3d(x, labels)
        else:
            fig, axes = plt.subplots(1, 1, figsize=(8,8))
            fig.suptitle(name)
            # Black removed and is used for noise instead.
            unique_labels = set(labels)
            colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
            core_samples_mask = np.zeros_like(labels, dtype=bool)
            if name == "DBSCAN":
                core_samples_mask[clustering.core_sample_indices_] = True
            for k, col in zip(unique_labels, colors):
                if k == -1:
                    # Black used for noise.
                    col = [0, 0, 0, 1]

                class_member_mask = (labels == k)

                xy = x[class_member_mask & core_samples_mask]
                axes.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                         markeredgecolor='k', markersize=12)

                xy = x[class_member_mask & ~core_samples_mask]
                axes.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                         markeredgecolor='k', markersize=6)
            plt.show()

In [ ]:
def elbow_knn(x, labels_true=None):
    inertias = []
    k_range = 15
    for k in range(1, k_range):
        model = KMeans(n_clusters=k, random_state=1)
        clustering = model.fit(x)
        labels=clustering.labels_
        inertias.append(clustering.inertia_)
    ## Results 
#         if labels_true:
#             print("Homogeneity: %0.3f" % homogeneity_score(labels_true, labels))
#             print("Completeness: %0.3f" % completeness_score(labels_true, labels))
#             print("V-measure: %0.3f" % v_measure_score(labels_true, labels))
#             print("Adjusted Rand Index: %0.3f" % adjusted_rand_score(labels_true, labels))
#             print("Adjusted Mutual Information: %0.3f" % adjusted_mutual_info_score(labels_true, labels))
#         if len(set(labels)) > 1:
#             print(f"Quality: {davies_bouldin_score(x, labels)}")
#             print("Silhouette Coefficient: %0.3f" % silhouette_score(x, labels))
#         print(f"n_clusters_: {len(set(labels)) - (1 if -1 in labels else 0)}")
#         print(f"n_noise_: {list(labels).count(-1)}")
    ## Plots
#         fig, axes = plt.subplots(1, 1, figsize=(8,8))
#         fig.suptitle(f"KNN: {k}")
#         # Black removed and is used for noise instead.
#         unique_labels = set(labels)
#         colors = [plt.cm.Spectral(each)
#                   for each in np.linspace(0, 1, len(unique_labels))]
#         core_samples_mask = np.zeros_like(labels, dtype=bool)
#         for k, col in zip(unique_labels, colors):
#             if k == -1:
#                 # Black used for noise.
#                 col = [0, 0, 0, 1]

#             class_member_mask = (labels == k)

#             xy = x[class_member_mask & core_samples_mask]
#             axes.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
#                      markeredgecolor='k', markersize=14)

#             xy = x[class_member_mask & ~core_samples_mask]
#             axes.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
#                      markeredgecolor='k', markersize=6)
#         plt.show()

    fig, ax = plt.subplots(1, 1, figsize=(8,8))
    fig.suptitle(f"KNN: inertia vs number of clusters")
    ax.plot(inertias)
    annot_elbow(range(1, k_range-1), inertias, ax)
    plt.show()

# Pętla aplikacji

In [ ]:
def main(columns):
    load_dataset()
    array = dataset.values
    x = dataset[columns].values
    x = StandardScaler().fit_transform(x)
#     labels_true = array[:,len(dataset.columns)-1]
    # noisy_circles = datasets.make_circles(n_samples=1500, factor=.5, noise=.05)
    # x,y = noisy_circles
    # labels_true = y
    elbow_knn(x)
    classify_each(x)

In [ ]:
json_config ="""
{
    "dataSourceUrl": "/home/jovyan/host-note/data/Mall_Customers.csv",
    "dataDelimiter": ",",
    "n_clusters": 5,
    "eps": 0.5,
    "min_samples": 9,
    "xi": 0.05,
    "min_cluster_size": 0.05,
    "m": 2,
    "ndims": 2
}"""
config = json.loads(json_config, object_hook = as_config)
main(['Inc','SpendScore'])

In [ ]:
json_config ="""
{
    "dataSourceUrl": "/home/jovyan/host-note/data/USArrests.csv",
    "dataDelimiter": ",",
    "n_clusters": 4,
    "eps": 0.1,
    "min_samples": 50,
    "xi": 0.05,
    "min_cluster_size": 0.05,
    "m": 2,
    "ndims": 2
}"""
config = json.loads(json_config, object_hook = as_config)
main(['Murder','Assault'])

In [ ]:
json_config ="""
{
    "dataSourceUrl": "/home/jovyan/host-note/data/StudentsPerformance.csv",
    "dataDelimiter": ",",
    "n_clusters": 5,
    "eps": 0.5,
    "min_samples": 9,
    "xi": 0.05,
    "min_cluster_size": 0.05,
    "m": 2,
    "ndims": 3
}"""
config = json.loads(json_config, object_hook = as_config)
main(['math score','reading score', 'writing score'])


In [ ]:
json_config ="""
{
    "dataSourceUrl": "/home/jovyan/host-note/data/cancer.csv",
    "dataDelimiter": ",",
    "n_clusters": 5,
    "eps": 0.5,
    "min_samples": 9,
    "xi": 0.05,
    "min_cluster_size": 0.05,
    "m": 2,
    "ndims": 3
}"""
config = json.loads(json_config, object_hook = as_config)
main(['radius_mean','texture_mean', 'area_mean'])